## Monte Carlo Influence Estimation
Notebook used for testing the social network enviroment and mainly used for the computation required in point 1 of the project.
Current implementation:
+ generates a random allocation for the ads of every category, basically does not take into account the bids yet
+ estimates the average number of seeds $|S|$ given the allocation
+ runs a Monte Carlo Influence Estimation algorithm with an increasing number of repetitions $R$
+ computes the error as: $ error = \sqrt{\frac{1}{R}\log (|S|) \log (\frac{1}{\sigma})} \cdot nodes$
with $\sigma = 0.05$
+ plots the error with respect to a specific MC run with a certain number of repetitions
+ plots the MC estimation of each run with its associated interval $\pm error$

### Currently Missing
+ generating the ads allocation through a real set of bids
+ citing the text: *"Use the best performance you obtain as a baseline and show the performance gap due a different number of iterations (also report the theoretical upper bound on the approximation error)"*

In [ ]:
import numpy as np 
from SocialNetwork.social_network_environment import *
from SocialNetwork.activation_probabilities import *
from SocialNetwork.influence_estimation import *


In [ ]:
enviroment = social_network_environment()
enviroment.init_from_json(json_path_network = 'Config/network.json', json_path_features = 'Config/features.json')

In [ ]:
import random
import json

adv_number = 8 # total advertisers for testing
ad_allocation = []
for i in range(5):
    ad_allocation.append(random.sample(range(adv_number), 6))

json_slots = open('Config/prominence.json', 'r')
slots = json.load(json_slots)


In [ ]:
seeds_num = estimate_seeds(social_network= enviroment, ad_allocation_list= ad_allocation,                                             learner_id=1, slot_prominence=slots['slots'], iterations= 1000)

In [ ]:

iterations = np.arange(start = 10, stop = 1000, step = 10)
activations = []

for iter in iterations:
    node_prob = estimate_bids_influence( social_network= enviroment, ad_allocation_list= ad_allocation,                                             learner_id=1, slot_prominence=slots['slots'], iterations= iter)
    net_percentage = np.mean(node_prob) * enviroment.active_nodes.shape[0]
    
    
    activations.append(net_percentage)
    print("ITER #",iter)



In [ ]:
import math

sigma = 0.05 # probability that the estimate is outside the confidence interval
nodes = enviroment.adj_matrix.shape[0] # number of nodes
S = seeds_num # average number of seeds given the set of bids

errors = []
for iter in iterations:
    error = (math.sqrt( 1/iter * math.log(S) * math.log(1/sigma)) * nodes)
    errors.append(error)


In [ ]:
from matplotlib import pyplot as plt

errors = np.array(errors)
activations = np.array(activations)

plt.figure(0)
plt.plot(iterations, errors, "green")
plt.xlabel("iterations")
plt.ylabel("error")

plt.figure(1)
plt.plot(iterations, activations, "red")
plt.xlabel("iterations")
plt.ylabel("active nodes")
plt.fill_between(iterations, (activations-errors), (activations+errors), color = "green", alpha = 0.2)



plt.show()


